# RAG评估指标
RAG的评估指标有很多种，总体上可以分为生成指标和检索指标。

生成指标则关注于最终产出的答案本身，衡量其是否忠实于检索到的内容、是否回答了用户的问题、以及语言表达是否自然。

检索指标衡量检索到的文档片段是否是高质量且与用户问题高度相关的。


## 生成指标
### 1. 忠实度
忠实度衡量的是模型生成的答案是否严格根据我们提供给它的上下文来回答问题，而没有自由发挥。 它的核心目标是量化并抑制模型的幻觉。

**评估方法：**

    1. 上下文：召回的上下文片段
    2. 生成的答案：大模型根据上下文生成的答案
    3. 忠实度评估：检查答案中的知识点是否都来做参考的上下文片段中。

**评估步骤：**

    1. 拆机答案：系统会将模型生成的长答案，拆分成一个个独立的事实陈述或**观点 **
    2. 逐一核查：接着，系统会拿着这每一个小陈述，回到给定的上下文中去寻找证据。
    3. 计算得分：最后，根据核查结果计算分数。忠实度 = 答案中被证实有依据的观点数量 / 答案中观点的总数量

### 2. 答案相关性

答案相关性指标衡量的是模型生成的答案 在多大程度上直接回应了用户提出的问题 。它的核心目标是确保答案既不“答非所问”，也不“言不及义”。不完整或包含冗余信息的答案将获得较低分数。

计算步骤
答案相关性的具体计算步骤如下：

逆向提问：系统会拿到模型生成的答案，然后反过来让LLM 为这个答案生成一个或多个最匹配的问题。这个过程会重复多次以确保公平。
使用的prompt:
Generate a question for the given answer.
answer: [answer]
这个步骤背后的逻辑是：如果回答完美地回应了用户问题，那么应该能从你的回答中精准地反推问题。
相似度比较：接下来，系统会使用余弦相似度算法，来比较“逆向生成的新问题”和“用户最初的原始问题”在语义上的相似程度。
计算得分：最后，将所有逆向生成问题与原始问题的相似度得分取一个平均值，就得出了最终的答案相关性分数。
举例说明
举个例子：

“谁是《三体》的作者？这部小说主要讲述了什么？”
现在看两个不同的答案：

答案A (高相关性)：“《三体》的作者是刘慈欣。这部小说主要讲述了地球人类文明和三体文明之间从信息交流、生死搏杀到两个文明在宇宙中兴衰历程的故事。”
核查结果: 这个答案直接、完整地回应了问题的两个部分（作者是谁，内容是什么）。从这个答案逆向生成的问题，很可能就是“《三体》的作者和内容是什么？”这与原始问题高度相似。

答案B (低相关性)：“《三体》的作者是刘慈欣。刘慈欣是中国当代著名的科幻作家，他还写过《流浪地球》等作品。”
核查结果: 这个答案只回答了第一个问题（作者是谁），但没有回答第二个问题（小说讲了什么），因此是不完整的。同时，它包含了“他还写过《流浪地球》”这类并未被问及的冗余信息。从这个答案逆向生成的问题，更可能是“刘慈欣是谁，他有哪些作品？”，这与原始问题的偏差就比较大了。

## 检索指标
### 1. 上下文精度

上下文精度专注于评估检索到的信息是否将最相关的内容排在了最前面。

通俗理解：评估检索召回的结果是否包含所有参考的片段，及片段的排列顺序

确定“标准答案”文档：首先，对于一个给定的问题，我们需要一个预先定义好的“标准答案”列表（ground-truth）。这个列表指明了哪些文档或数据块是回答该问题的最相关的来源。
评估检索结果的排序：查看RAG系统返回的上下文列表，并检查其中包含了多少“标准答案”文档，以及它们分别排在第几位（rank）。
计算得分： 该指标奖励排名靠前的相关文档。排名越高的文档贡献的分数越高，例如，第1名贡献1分，第2名贡献1/2分，第3名贡献1/3分……。最后将所有找到的“标准答案”文档的得分相加再取平均值。

举例说明
假设我们有一个关于AI框架的知识库。

问题 ：“关于RAG，LangChain和LlamaIndex有什么主要区别？”
“标准答案”文档：我们预先知道，回答这个问题的最佳文档是文档A(对比框架)、文档C(LangChain的核心功能) 和文档F (LlamaIndex的索引机制)。
现在，我们来看两种不同的检索结果：

情况A (高精度)

检索到的上下文 (Contexts)：[文档A, 文档D, 文档F, 文档B, 文档C]
核查结果：
文档A(标准答案) 位于第1位。
文档F(标准答案) 位于第3位。
文档C(标准答案) 位于第5位。

精度得分：由于大部分关键文档都排在列表的靠前位置，得分会比较高。
情况B (低精度)

检索到的上下文 (Contexts)：[文档B, 文档E, 文档D, 文档G, 文档A]
核查结果：
文档A(标准答案) 位于第5位。
文档C和文档F(标准答案)甚至没有被检索到。

精度得分：由于关键文档排名非常靠后，且有遗漏，最终的精度得分会非常低。


### 2. 上下文召回率(context Recall)

上下文召回率主要衡量我们检索到的上下文在多大程度上覆盖了回答问题所需的全部信息。换言之，它关心的是“我们找来的资料，够不够用？”。

**通俗理解：**用户问题涉及4个文档片段[A, B, C, D]，检索系统召回的文档片段是否都包含这4个。


### 3. 上下文相关性 (Context Relevancy)
上下文相关性衡量的是，我们检索到的上下文在多大程度上是专门针对用户问题的。它的核心目标是评估检索系统能否提供“信噪比”高的信息，避免无关内容对后续的答案生成造成干扰。

通俗理解：系统召回的结果中和问题相关的上下文片段的比例

计算步骤
上下文相关性的具体计算步骤如下：

拆解上下文：首先，系统会将检索到的整个上下文拆分成独立的句子。得到上下文中的总句子数。
逐句甄别：接着，系统会逐一检查每个句子，并判断：“这个句子对于回答最初的问题来说，是相关的吗？”
计算得分：最后，统计出所有被判定为“相关”的句子数量，并根据以下公式计算最终得分：
**上下文相关性=上下文中与问题相关的句子数量 / 上下文中的总句子数量**
这个结果在 0 到 1 之间的分数。分数越接近 1，代表检索到的上下文相关性越高，包含的无用信息越少。


# 评估方法 - 使用RAGRS评估
使用RAGS进行评估

In [ ]:
print('ok')

#